# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [98]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bond data

In [99]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2021-12-30 18:28:19.808 | INFO     | src.data.make_dataset:get_bond_data:34 - Load bond data
2021-12-30 18:28:19.809 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\bonds.csv


In [100]:
df_bonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                225 non-null    string        
 1   country            225 non-null    string        
 2   issue_dt           225 non-null    datetime64[ns]
 3   first_coupon_date  202 non-null    datetime64[ns]
 4   mature_dt          225 non-null    datetime64[ns]
 5   isin               225 non-null    string        
 6   issuer_name        225 non-null    string        
 7   coupon_frq         225 non-null    string        
 8   coupon             225 non-null    float64       
 9   tot_issue          225 non-null    float64       
 10  cfi_code           218 non-null    string        
 11  issue_rating       220 non-null    string        
dtypes: datetime64[ns](3), float64(2), string(7)
memory usage: 21.2 KB


In [101]:
df_bonds.head()

,ccy,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,coupon_frq,coupon,tot_issue,cfi_code,issue_rating
0,EUR,Netherlands,2009-02-13,2009-07-15,2019-07-15,NL0009086115,STAAT DER NEDERLANDEN,ANNUAL,4.000,5.000000e+09,DBFTFN,AAA
1,NLG,Austria,1994-02-28,1995-02-28,2024-02-28,NL0000133924,AUSTRIA,ANNUAL,6.250,1.000000e+09,DBFTXB,AA+
2,EUR,Netherlands,2012-03-09,2013-01-15,2033-01-15,NL0010071189,STAAT DER NEDERLANDEN,ANNUAL,2.500,4.160000e+09,DBFXXN,AAA
3,USD,United States,2009-05-15,2009-11-15,2019-05-15,US912828KQ20,UNITED STATES TREASURY,SEMI ANNUAL,3.125,6.441100e+10,<NA>,<NA>
4,USD,United States,2010-02-15,2010-08-15,2020-02-15,US912828MP29,UNITED STATES TREASURY,SEMI ANNUAL,3.625,0.000000e+00,<NA>,<NA>


Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [102]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2021-12-30 18:28:31.282 | INFO     | src.data.make_dataset:impute_bonds:67 - Impute bond data


In [103]:
make_dataset.save_pkl('bonds', df_bonds)

2021-12-30 18:28:32.610 | INFO     | src.data.make_dataset:save_pkl:290 - Save preprocessed bonds data


# Bondprijzen

In [109]:
df_price = make_dataset.get_price()

2021-12-30 18:32:22.546 | INFO     | src.data.make_dataset:get_price:104 - Load bond price data
2021-12-30 18:32:22.547 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\price.csv


In [111]:
df_price = make_dataset.impute_price(df_price)

2021-12-30 18:32:29.017 | INFO     | src.data.make_dataset:impute_price:133 - Impute bond price


In [112]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601


In [113]:
df_price.describe()

,mid
count,209276.000000
mean,111.289025
std,15.522802
min,84.429000
25%,102.285000
50%,106.729000
75%,112.926000
max,195.749000


In [114]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209276 entries, 0 to 220236
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  209276 non-null  string        
 1   ccy                   209276 non-null  string        
 2   rate_dt               209276 non-null  datetime64[ns]
 3   mid                   209276 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.0 MB


In [115]:
make_dataset.save_pkl('price', df_price)

2021-12-30 18:32:37.117 | INFO     | src.data.make_dataset:save_pkl:294 - Save preprocessed price data


# Government Yield curves

In [116]:
df_yield = make_dataset.get_yield()

2021-12-30 18:32:38.583 | INFO     | src.data.make_dataset:get_yield:141 - Load goverment yield curve data
2021-12-30 18:32:38.583 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\yield.csv


In [117]:
df_yield = make_dataset.impute_yield(df_yield)

2021-12-30 18:32:40.573 | INFO     | src.data.make_dataset:impute_yield:166 - Impute yield curve


In [118]:
df_yield.head()

,ratename,ccy,rate_dt,timeband,actual_dt,datedays,bid,offer,int_basis,country
0,GOV Yield Curve DE BB,EUR,2017-02-07,1 YEAR,2018-02-09,367,-0.875,-0.804,ANNUAL,Germany
1,GOV Yield Curve DE BB,EUR,2017-02-07,2 YEARS,2019-02-11,734,-0.799,-0.773,ANNUAL,Germany
2,GOV Yield Curve DE BB,EUR,2017-02-07,3 YEARS,2020-02-10,1098,-0.769,-0.754,ANNUAL,Germany
3,GOV Yield Curve DE BB,EUR,2017-02-07,4 YEARS,2021-02-09,1463,-0.620,-0.602,ANNUAL,Germany
4,GOV Yield Curve DE BB,EUR,2017-02-07,5 YEARS,2022-02-09,1828,-0.414,-0.404,ANNUAL,Germany


In [119]:
df_yield.describe()

,datedays,bid,offer
count,89824.000000,89824.000000,89824.000000
mean,3389.145162,0.284556,0.274044
std,2877.661657,0.921327,0.921680
min,367.000000,-1.038000,-1.051000
25%,1463.000000,-0.429000,-0.443000
50%,2560.000000,0.016000,0.009000
75%,3657.000000,0.756000,0.750000
max,10968.000000,4.128000,4.109000


# Inflation data


In [120]:
df_inflation = make_dataset.get_inflation()  

2021-12-30 18:32:45.955 | INFO     | src.data.make_dataset:get_inflation:178 - Load goverment yield curve data
2021-12-30 18:32:45.956 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\DE Inflation.csv
2021-12-30 18:32:45.977 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\FR Inflation.csv
2021-12-30 18:32:45.994 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\ES Inflation.csv
2021-12-30 18:32:46.011 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\IT Inflation.csv
2021-12-30 18:32:46.030 | INFO     | src.data.make_dataset:read_csv:19 - Loading data from ..\data\raw\US Inflation.csv


In [121]:
df_inflation = make_dataset.impute_inflation(df_inflation)

df_inflation.info()

2021-12-30 18:33:00.779 | INFO     | src.data.make_dataset:impute_inflation:205 - Impute inflation curve


<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   country    215007 non-null  string        
 1   rate_dt    215007 non-null  datetime64[ns]
 2   timeband   215007 non-null  string        
 3   inflation  215007 non-null  float64       
 4   ratename   215007 non-null  string        
dtypes: datetime64[ns](1), float64(1), string(3)
memory usage: 9.8 MB


In [59]:
#df_price.merge(df_bonds, left_on = 'reference_identifier', right_on='isin',  how = 'left')

In [126]:
df_bonds.count()

ccy                  225
country              225
issue_dt             225
first_coupon_date    225
mature_dt            225
isin                 225
issuer_name          225
coupon_frq           225
coupon               225
tot_issue            225
cfi_code             225
issue_rating         225
bond_duration        225
dtype: int64

In [127]:
df = make_dataset.join_full(df_bonds,df_price,df_yield, df_inflation)



number of bonds  2592
number of prices  837104
number of df  3348416


In [128]:
df.count()

reference_identifier    209276
ccy                     209276
rate_dt                 209276
mid                     209276
country                 209276
issue_dt                209276
first_coupon_date       209276
mature_dt               209276
isin                    209276
issuer_name             209276
coupon_frq              209276
coupon                  209276
tot_issue               209276
cfi_code                209276
issue_rating            209276
bond_duration           209276
dtype: int64

In [95]:
df = df_price[df_price['reference_identifier'] == 'BE0000332412']

,reference_identifier,ccy,rate_dt,mid
2225,BE0000332412,EUR,2019-07-30,115.638


In [ ]:
df_inflation.describe()

All in one make statement...

In [ ]:
make_dataset.make_data()